## Unsupervised training data generation

In [ ]:
# builtins
import pathlib
from logging.config import dictConfig

# externals
import numpy as np
from osgeo import gdal
import matplotlib.pyplot as plt
import xarray as xr

# locals
from ai4ebv.core.sample import TrainingDataFactory
from ai4ebv.core.legend import Legend
from ai4ebv.core.dataset import HLSDataset
from ai4ebv.core.constants import STT_TILES
from ai4ebv.core.utils import mosaic_tiles
from ai4ebv.main.io import *
from pysegcnn.core.utils import img2np, np2tif
from pysegcnn.core.logging import log_conf

In [ ]:
dictConfig(log_conf())

### Removal of boundary pixels

In [ ]:
layer = pathlib.Path('/mnt/CEPH_PROJECTS/AI4EBV/DISSEMINATION/ESA-Phi-week-2021/Landcover/CORINE/Mosaic.tif')

In [ ]:
# original land cover mask
lc = img2np(layer)
np2tif(lc, filename='./Figures/no_erosion.tif', no_data=Legend.NoData.id, src_ds=gdal.Open(str(layer)), overwrite=True)

In [ ]:
# eroded land cover mask
lc_eroded = np.ones(lc.shape, dtype=np.int16) * Legend.NoData.id
for label in Legend:
    lc_eroded[TrainingDataFactory.is_homogeneous(lc == label.id)] = label.id

In [ ]:
# save eroded mask
np2tif(lc_eroded, filename='./Figures/erosion.tif', no_data=Legend.NoData.id, src_ds=gdal.Open(str(layer)), overwrite=True)

### Spectral filter

In [ ]:
for tile in STT_TILES:
    # instanciate the HLS-dataset
    hls = HLSDataset.initialize(HLS_PATH, tile, 2018, months=[3,4,5,6,7,8,9])
    
    # load time series
    hls_ts = hls.to_xarray(spatial_coverage=80, cloud_coverage=50, save=False)
    
    # instanciate training data factory
    factory = TrainingDataFactory(hls_ts, CLC_LAYERS[tile], 'CORINE')
    
    # save filtered pixels
    np2tif(factory.samples, filename='./Figures/samples_{}.tif'.format(tile), no_data=Legend.NoData.id,
           src_ds=gdal.Open(str(CLC_LAYERS[tile])), overwrite=True)

In [ ]:
# mosaic filtered pixels
mosaic = mosaic_tiles([pathlib.Path('./Figures/samples_{}.tif'.format(tile)) for tile in STT_TILES], targets='./Figures/samples.tif', no_data=Legend.NoData.id)